In [ ]:
#CNN-BiLSTM-GRU (Phase Space Reconstruction) 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPooling1D, GRU
import matplotlib.pyplot as plt# Load data

# Load data
data = pd.read_csv('WMT.csv')

# Preprocessing
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

# Define a new scaler for the 'Adj Close' column
scaler_adj_close = MinMaxScaler()
data_normalized['Adj Close'] = scaler_adj_close.fit_transform(data[['Adj Close']])

# Create the sequences using time delay embedding
delay = 1
embedding_dim = 3
X, Y = [], []
for i in range(embedding_dim * delay, len(data_normalized)):
    X.append(data_normalized.iloc[i - embedding_dim * delay:i:delay].values.reshape(embedding_dim, -1))
    Y.append(data_normalized.iloc[i, 5])
X, Y = np.array(X), np.array(Y)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = Y[:train_size], Y[train_size:]

# Define the model
model = Sequential()

# Add a 1D Convolutional layer with 64 filters
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(embedding_dim, data_normalized.shape[1])))

# Add a Bidirectional LSTM layer with 50 units
model.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True)))

# Add a GRU layer
model.add(GRU(50))

# Add a Dense layer with 1 unit
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(trainX, trainY, epochs=50, validation_data=(testX, testY), verbose=1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict on the test data
preds = model.predict(testX)

# Un-normalize the predicted values
preds_unscaled = scaler_adj_close.inverse_transform(preds)

# Un-normalize the actual values
actual_unscaled = scaler_adj_close.inverse_transform(testY.reshape(-1, 1))

# Calculate and print MSE and MAE
mse = mean_squared_error(actual_unscaled, preds_unscaled)
mae = mean_absolute_error(actual_unscaled, preds_unscaled)
rmse = np.sqrt(mean_squared_error(actual_unscaled, preds_unscaled))
print(f'MSE: {mse}, MAE: {mae}, RMSE: {rmse}')

In [ ]:
#CNN-BiLSTM-GRU (Standard looback of 5 data points) 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPooling1D, GRU
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('WMT.csv')

# Preprocessing
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

# Define a new scaler for the 'Adj Close' column
scaler_adj_close = MinMaxScaler()
data_normalized['Adj Close'] = scaler_adj_close.fit_transform(data[['Adj Close']])

# Create the sequences
lookback = 5
X, Y = [], []
for i in range(lookback, len(data_normalized)):
    X.append(data_normalized.iloc[i-lookback:i].values)
    Y.append(data_normalized.iloc[i, 0])
X, Y = np.array(X), np.array(Y)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = Y[:train_size], Y[train_size:]

# Define the model
model = Sequential()

# Add a 1D Convolutional layer with 64 filters
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(lookback, data_normalized.shape[1])))

# Add a MaxPooling layer
model.add(MaxPooling1D(pool_size=2))

# Add a Bidirectional LSTM layer with 50 units
model.add(Bidirectional(LSTM(50, activation='relu', return_sequences=True)))

# Add a GRU layer
model.add(GRU(50))

# Add a Dense layer with 1 unit
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(trainX, trainY, epochs=50, validation_data=(testX, testY), verbose=1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict on the test data
preds = model.predict(testX)

# Un-normalize the predicted values
preds_unscaled = scaler_adj_close.inverse_transform(preds)

# Un-normalize the actual values
actual_unscaled = scaler_adj_close.inverse_transform(testY.reshape(-1, 1))

# Calculate and print MSE and MAE
mse = mean_squared_error(actual_unscaled, preds_unscaled)
mae = mean_absolute_error(actual_unscaled, preds_unscaled)
rmse = np.sqrt(mean_squared_error(actual_unscaled, preds_unscaled))
print(f'MSE: {mse}, MAE: {mae}, RMSE: {rmse}')